<a href="https://colab.research.google.com/github/yseeker/pytorch-templates/blob/main/template_Tez_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Jul  2 03:19:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install gwpy --quiet

     |████████████████████████████████| 1.4MB 15.9MB/s 
     |████████████████████████████████| 10.3MB 33.2MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 3.2MB 46.7MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
%%capture
!pip install wandb

class CFG:
    project_name = 'project name'
    pretrained_model_name = 'vgg16_bn'
    lr = 5e-4
    batch_size= 16
    wandb_note = f'bs{batch_size}_adamW_default_lr{lr}'
    pretrained = True
    prettained_path = '../input/timm_weight/efficientnet_v2s_ra2_288-a6477665.pth'
    input_channels = 3
    out_dim = 1
    colab_or_kaggle = 'colab'
    wandb_exp_name = f'{pretrained_model_name}_{colab_or_kaggle}_{wandb_note}'
    monitor = 'valid_roc_auc'
    epochs = 3
    num_of_fold = 5
    seed = 42
    num_workers = 8
    fp16 = True
    checkpoint_path = ''
    patience_mode = 'max'
    patience = 10
    delta = 0.001
    mixup_alpha = 1.0
    benchmark = False

# AdamW, VGG16, only0.6

In [ ]:
# !unzip "/content/drive/MyDrive/kaggle/input/***/data.zip" -d "/content"

In [ ]:
import os
import sys
import random
from tqdm import tqdm
import math

import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
import torchvision

import cv2
from PIL import Image
import albumentations as A

In [ ]:
#!pip install timm
os.chdir("/content/drive/My Drive/kaggle/working")
tez_path = '../input/tez-lib'
#efnet_path = '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'
timm_path = '../input/pytorch-image-models-master'
sys.path.append(tez_path)
#sys.path.append(efnet_path)
sys.path.append(timm_path)


#from efficientnet_pytorch import model as enet
import tez
from tez.datasets import ImageDataset
from tez.callbacks import EarlyStopping
import timm

In [ ]:
train_aug = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(p=0.5,
                           shift_limit = 0.2, 
                           scale_limit=0.2,
                           rotate_limit=30, 
                           border_mode = cv2.BORDER_REPLICATE),
        A.OneOf([
            A.MedianBlur(p=0.3),
            A.MotionBlur(p=0.3)
        ]
        )
    ]
)
df = pd.read_csv('../input/***/train_labels.csv')
df['img_path'] = df['id'].apply(
    lambda x: f'../input/***/train/{x[0]}/{x}.npy'
)

X = df.img_path.values
Y = df.target.values

skf = StratifiedKFold(n_splits = CFG.num_of_fold)

In [ ]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

# define vectorized sigmoid
sigmoid_v = np.vectorize(sigmoid)

def set_seed(seed = 0):
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state

def mixup_data(inputs, targets, alpha=1.0):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = inputs.size()[0]
    index = torch.randperm(batch_size)
    mixed_inputs = lam * inputs + (1 - lam) * inputs[index, :]
    targets_a, targets_b = targets, targets[index]
    
    return mixed_inputs, targets_a, targets_b, lam

def mixup_criterion(criterion, outputs, targets_a, targets_b, lam):
    return lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)

In [ ]:
class ClassificationDataset():
    def __init__(self, image_paths, targets, transform = None): 
        self.image_paths = image_paths
        self.targets = targets
        self.transform = None

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item): 
        targets = self.targets[item]
        image = np.load(self.image_paths[item])
        image = image[np.newaxis, ]

        return torch.tensor(image, dtype=torch.float), torch.tensor(targets, dtype=torch.float)


class CustomNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model(CFG.pretrained_model_name, 
                                       pretrained = CFG.pretrained, 
                                       in_chans = CFG.input_channels)
        if not CFG.pretrained: self.model.load_state_dict(torch.load(CFG.pretrained_path))
        self.model.classifier = nn.Linear(self.model.classifier.in_features, CFG.out_dim)
        
    def forward(self, inputs):
        outputs = self.model(inputs)
        return outputs


class Trainer(tez.Trainer):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.criterion =  nn.BCEWithLogitsLoss()

    def epoch_metrics(self, outputs, targets):
        outputs = sigmoid_v(outputs)
        roc_auc = metrics.roc_auc_score(targets, outputs)
        return roc_auc

    def monitor_metrics(self, outputs, targets):
        outputs = outputs.sigmoid().cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        if len(np.unique(targets)) > 1: 
            roc_auc = metrics.roc_auc_score(targets, outputs)
        else: roc_auc = 0.5
        return {"roc_auc": roc_auc}

    def configure_optimizer(self):
        opt = torch.optim.AdamW(self.parameters(), lr=CFG.lr, weight_decay=0.01)
        #opt = torch.optim.SGD(self.parameters(), lr=0.001, momentum=0.9)
        return opt
    
    def configure_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        return sch

In [ ]:
models = []
for fold_cnt, (train_index, test_index) in enumerate(skf.split(X, Y)):
    train_images, valid_images = X[train_index], X[test_index]
    train_targets, valid_targets = Y[train_index], Y[test_index]

    train_dataset = ClassificationDataset(
        image_paths=train_images, 
        targets=train_targets, 
        transform = train_aug
    )
    valid_dataset = ClassificationDataset(
        image_paths=valid_images, 
        targets=valid_targets, 
        transform = None
    )

    es = EarlyStopping(
        monitor=CFG.monitor, 
        model_path=CFG.checkpoint_path+f'{CFG.pretrained_model_name}_{CFG.wandb_note}.cpt', 
        patience= CFG.patience, 
        mode=CFG.patience_mode,
        delta = CFG.delta
    )
    model = CustomNN()

    trainer = Trainer(model = model)

    trainer.fit(
        cfg = CFG,
        train_dataset = train_dataset,
        valid_dataset = valid_dataset,
        valid_targets = valid_targets,
        train_bs=CFG.batch_size,
        valid_bs=CFG.batch_size,
        epochs=CFG.epochs,
        callbacks=[es],
        n_jobs = CFG.num_workers,
        fp16=CFG.fp16,
        benchmark = CFG.benchmark
    )

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


KeyboardInterrupt: ignored

In [ ]:
!cp "/content/drive/MyDrive/kaggle/input/***/sample_submission.csv" "/content"
!unzip "/content/drive/MyDrive/kaggle/input/***/data.zip" -d "/content/test"

In [ ]:
submission = pd.read_csv('/content/sample_submission.csv')
submission['img_path'] = submission['id'].apply(
    lambda x: f'/content/test/{x}.npy'
)
test_dataset = ClassificationDataset(
    image_paths=submission.img_path.values, 
    targets=submission.target.values, 
    transform = train_aug
)

final_preds = None
num_of_ave = 5
outs = []
for model in models:
    for j in range(num_of_ave):
        preds = model.predict(test_dataset, batch_size=128, n_jobs=-1)
        temp_preds = None
        for p in preds:
            if temp_preds is None:
                temp_preds = p
            else:
                temp_preds = np.vstack((temp_preds, p))
        if final_preds is None:
            final_preds = temp_preds
        else:
            final_preds += temp_preds
    final_preds = final_preds/num_of_ave
    out = sigmoid_v(final_preds)
    outs.append(out)
pred = np.mean(np.array(outs), axis=0)
submission.target = pred
submission.drop(['img_path'], axis=1, inplace=True)
submission.to_csv('submission.csv', index=False)

In [ ]:
# for kaggle
import os
import json
f = open("/content/drive/My Drive/kaggle/kaggle.json", 'r')
json_data = json.load(f) #JSON形式で読み込む
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']
os.chdir("/content/drive/My Drive/kaggle/working")

!kaggle competitions submit -c titanic -f submission.csv -m 'submit!!'